# Method 1: Spatial microsimulation

---

### Step 1: Importing all the necessary packages for our code

In [39]:
import pandas as pd
import numpy as np
import copy
from ipfn import ipfn
import matplotlib.pyplot as plt


### Step 2: Importing our individual-level BSA data and our constraints

In [3]:
BSA_2020 = pd.read_csv('data/BSA2020/final_bsa_cleaned.csv', low_memory=False)
con_age = pd.read_csv('data/census data/con_age.csv')
con_sex = pd.read_csv('data/census data/con_sex.csv') # 1 represents female, 2 represents male

We also need to check our constraints to ensure that they both have the same population.

In [4]:
print('con_age total: ',con_age['age_1864'].sum() + con_age['age_65'].sum())
print('con_sex total: ',con_sex['sex_male'].sum() + con_sex['sex_female'].sum())

con_age total:  162096
con_sex total:  162096


The above calculations show that both constraints represent the same population size for adults in York.

### Step 3: Preparing the data for spatial microsimulation
First, we will remove all of the BSA_2020 data that isn't related to out constraints. We only need age and sex to be able to allocate individuals to zones so we'll copy the data set and remove all other columns.

In [5]:
BSA_original = BSA_2020.copy() # making a deep copy

In [6]:
# dropping all columns unnecessary for weighting
BSA_2020 = BSA_2020.drop(['economic', 
                          'partySup', 
                          'partySupWho',
                          'polInterest',
                          'welfare',
                          'redistrb',
                          'leftright',
                          'leftright2',
                          'libauth',
                          'libauth2',
                          'religion',
                          'nationality',
                          'raceOrigin',
                          'disability',
                          'voteAct',
                          'voteParty'
                         ],axis=1)

Next we need to recategorise the BSA_2020 individual data into bins that match the categories in our constraints dataframe.

In [7]:
BSA_2020['age'] = pd.to_numeric(BSA_2020['age']) # ensuring age is numeric

# changing age to categorical attribute, with categories matching the indexing of the age constraint
BSA_2020['age'] = pd.cut(BSA_2020['age'], [17,64,120], labels = ['age_1864','age_65'])
BSA_2020.head()

,id,age,sex
0,1000,age_1864,2
1,1001,age_65,1
2,1002,age_65,1
3,1003,age_1864,2
4,1004,age_1864,1


Let's do the same with the sex constraint, and rename the categories in the constraints dataframe to match

In [8]:
BSA_2020['sex'] = pd.to_numeric(BSA_2020['sex'])

# changing age to categorical attribute, with categories matching the indexing of the age constraint
BSA_2020['sex'] = pd.cut(BSA_2020['sex'], [0,1,2], labels = ['f','m'])
BSA_2020.head()

,id,age,sex
0,1000,age_1864,m
1,1001,age_65,f
2,1002,age_65,f
3,1003,age_1864,m
4,1004,age_1864,f


In [9]:
BSA_2020['age'].unique()

['age_1864', 'age_65']
Categories (2, object): ['age_1864' < 'age_65']

In [10]:
con_sex = con_sex.rename(columns={'sex_female':'f','sex_male':'m'}) # all working and looking good!

Next we'll merge these constraints into one table together, merging it on geo_code to avoid duplicate columns and to match up the zones.

In [11]:
constraints = con_sex.merge(con_age, left_on='geo_code', right_on='geo_code', how='right')

Now that's all merged, but for the next step we also need to drop the geo_code column from the dataframe. We'll make a copy of the constraints dataframe as it is now and then drop the geo_code column from the dataframe we're using.

In [12]:
constraints_geo = constraints.copy() # making a deep copy

In [13]:
constraints = constraints.drop(columns=['geo_code'])

In [14]:
constraints.head()

,m,f,age_1864,age_65
0,3229,3507,5183,1553
1,1497,1682,2179,1000
2,5295,5818,9413,1700
3,1381,1533,1869,1045
4,4250,4723,6665,2308


This gives us one full table with all of our constaint variables, with the names of columns matching the individual level data we have in our BSA_2020 table. The next step to prep the data is to 'flatten' the individual level dataset, so it can be more easily compared with the constraints. This basically means that we turn the data into fields, and have the data entries themselves become boolean. We do this first individually for age and then for sex, and then we combine these into one dataframe.

In [15]:
# Checking the dimensions of our two dataframes
print ("Shape of our BSA dataframe:",BSA_2020.shape)
print ("Shape of our constraints:",constraints.shape)

Shape of our BSA dataframe: (275, 3)
Shape of our constraints: (22, 4)


So we need these to have the same number of columns. At this time, our BSA dataframe has three columns and our constraints has four.

In [16]:
# flattening the age column
age_flat = pd.pivot_table(BSA_2020,columns=['age'],values='id', index=BSA_2020.index, aggfunc=len, fill_value=0, observed=False)

# flattening the sex column
sex_flat = pd.pivot_table(BSA_2020,columns=['sex'],values='id', index=BSA_2020.index, aggfunc=len, fill_value=0, observed=False)

In [17]:
# merge pivoted data to make flatten dataframe
bsa_categorical = pd.DataFrame(age_flat.to_records()).merge(pd.DataFrame(sex_flat.to_records()),left_index=True,right_index=True)

# the above line creates a dataframe with extra index columns, we don
bsa_categorical = bsa_categorical.drop(['index_x','index_y'],axis=1)
bsa_categorical

,age_1864,age_65,f,m
0,1,0,0,1
1,0,1,1,0
2,0,1,1,0
3,1,0,0,1
4,1,0,1,0
...,...,...,...,...
270,1,0,1,0
271,1,0,1,0
272,0,1,1,0
273,1,0,1,0


In [18]:
# checking the column sums to ensure that this is correct and saving these values for later
bsa_agg = bsa_categorical.sum(axis=0)
bsa_agg # age adds up to 275 (total number of individual data entries) and so does sex so it's worked!

age_1864    192
age_65       83
f           155
m           120
dtype: int64

In [19]:
print ("Shape of individual flattened dataframe:",bsa_categorical.shape)
print ("Shape of constraints dataframe:",constraints.shape)

Shape of individual flattened dataframe: (275, 4)
Shape of constraints dataframe: (22, 4)


As we can see when the above code is run, our two dataframe now have the same dimensions and can be compared, as is demonstrated by the code below. The test dataframe created below shows us the first row of the constaints dataframe and the first row of the aggregated dataframe and confirms that both dataframes share the same dimesions and column names.

In [20]:
test = pd.concat([constraints.iloc[0], bsa_agg], axis=1).transpose()
test

,m,f,age_1864,age_65
0,3229,3507,5183,1553
0,120,155,192,83


Now our data is all prepped, we can start working towards applying the microsimulation methods to create our synthetic population. 

### Step 4: Iterative Proportional Fitting

Whilst a lot of the coding for IPF can be done with python packages that we can import and use, the general gist of it is to generate weights to show how representative each individual is of each zone.

In [21]:
BSA_2020_copy = BSA_2020.copy() # making another deep copy of BSA_2020 with the changes we made to variables
BSA_2020_copy

,id,age,sex
0,1000,age_1864,m
1,1001,age_65,f
2,1002,age_65,f
3,1003,age_1864,m
4,1004,age_1864,f
...,...,...,...
270,1271,age_1864,f
271,1272,age_1864,f
272,1273,age_65,f
273,1274,age_1864,f


In [22]:
BSA_2020_copy.info() # looking at data types - we have two category types

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275 entries, 0 to 274
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   id      275 non-null    int64   
 1   age     275 non-null    category
 2   sex     275 non-null    category
dtypes: category(2), int64(1)
memory usage: 3.1 KB


In [23]:
# changing the categorical data into strings - the package we're using doesn't like category data!
BSA_2020_copy['age'] = BSA_2020_copy['age'].astype(str)
BSA_2020_copy['sex'] = BSA_2020_copy['sex'].astype(str)
BSA_2020_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275 entries, 0 to 274
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      275 non-null    int64 
 1   age     275 non-null    object
 2   sex     275 non-null    object
dtypes: int64(1), object(2)
memory usage: 6.6+ KB


Now our BSA_2020_copy data is in the correct datatype, we can use a for loop to add columns with the caluculated weights. Increasing the efficiency from the example provided within the IPF guide page, we can do this with just four lines of code.
1. The first line initiates the loop, and instructs it to repeat for the length of the contstraints dataframe (the number of electoral zones in York).
2. The second line (and the first within the for loop) creates a new column (data-type float) containing 1.0 for each individual to represent the weighting for the current zone.
3. The third carries out the IPF analysis, the key inputs being the dataframe itself, the aggregates (which don't need to be predefined as they can be defined within this line), and the dimensions (the column names we are working with).
4. The final line adds the results of the previous calculation into our BSA_2020_copy dataframe.

In [24]:
for i in range(0,len(constraints)):
    # creates a column with float value 1.0 to hold that zone's weight
    BSA_2020_copy['weight_' + str(i)] = np.ones(275) 
    
    # carries out the ipf analysis using the dataframe, the aggregates (the constraints database), and the dimensions (sex and age)
    ipf = ipfn.ipfn(BSA_2020_copy, [constraints.iloc[i,[0,1]], constraints.iloc[i,[2,3]]],[['sex'],['age']],weight_col='weight_'+str(i),convergence_rate = 1e-15)
    
    BSA_2020_copy = ipf.iteration()

ipfn converged: convergence_rate not updating or below rate_tolerance
ipfn converged: convergence_rate not updating or below rate_tolerance
ipfn converged: convergence_rate not updating or below rate_tolerance
ipfn converged: convergence_rate not updating or below rate_tolerance
ipfn converged: convergence_rate not updating or below rate_tolerance
ipfn converged: convergence_rate not updating or below rate_tolerance
ipfn converged: convergence_rate not updating or below rate_tolerance
ipfn converged: convergence_rate not updating or below rate_tolerance
ipfn converged: convergence_rate not updating or below rate_tolerance
ipfn converged: convergence_rate not updating or below rate_tolerance
ipfn converged: convergence_rate not updating or below rate_tolerance
ipfn converged: convergence_rate not updating or below rate_tolerance
ipfn converged: convergence_rate not updating or below rate_tolerance
ipfn converged: convergence_rate not updating or below rate_tolerance
ipfn converged: conv

In [25]:
BSA_2020_copy # look at all those weights we've calculated!

,age,sex,id,weight_0,weight_1,weight_2,weight_3,weight_4,weight_5,weight_6,...,weight_12,weight_13,weight_14,weight_15,weight_16,weight_17,weight_18,weight_19,weight_20,weight_21
0,age_1864,m,1000,30.705676,12.289888,56.887145,10.449594,38.612621,47.343966,8.789196,...,53.887440,51.003342,38.482515,57.421184,11.920063,34.641819,49.529351,29.615779,48.801957,14.259688
1,age_65,f,1001,16.498045,11.177411,17.671996,11.782094,25.103857,11.047300,7.847097,...,17.804158,11.059227,34.189175,14.226022,9.710651,24.793063,22.071087,14.110091,23.267020,8.170437
2,age_65,f,1002,16.498045,11.177411,17.671996,11.782094,25.103857,11.047300,7.847097,...,17.804158,11.059227,34.189175,14.226022,9.710651,24.793063,22.071087,14.110091,23.267020,8.170437
3,age_1864,m,1003,30.705676,12.289888,56.887145,10.449594,38.612621,47.343966,8.789196,...,53.887440,51.003342,38.482515,57.421184,11.920063,34.641819,49.529351,29.615779,48.801957,14.259688
4,age_1864,f,1004,24.616020,10.745798,43.986873,9.275901,32.214132,31.668398,7.801797,...,39.738820,34.878200,34.810354,40.644540,10.017054,29.802253,39.250416,23.135184,40.947464,11.286525
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270,age_1864,f,1271,24.616020,10.745798,43.986873,9.275901,32.214132,31.668398,7.801797,...,39.738820,34.878200,34.810354,40.644540,10.017054,29.802253,39.250416,23.135184,40.947464,11.286525
271,age_1864,f,1272,24.616020,10.745798,43.986873,9.275901,32.214132,31.668398,7.801797,...,39.738820,34.878200,34.810354,40.644540,10.017054,29.802253,39.250416,23.135184,40.947464,11.286525
272,age_65,f,1273,16.498045,11.177411,17.671996,11.782094,25.103857,11.047300,7.847097,...,17.804158,11.059227,34.189175,14.226022,9.710651,24.793063,22.071087,14.110091,23.267020,8.170437
273,age_1864,f,1274,24.616020,10.745798,43.986873,9.275901,32.214132,31.668398,7.801797,...,39.738820,34.878200,34.810354,40.644540,10.017054,29.802253,39.250416,23.135184,40.947464,11.286525


In [37]:
BSA_2020_copy.head()

,age,sex,id,weight_0,weight_1,weight_2,weight_3,weight_4,weight_5,weight_6,...,weight_12,weight_13,weight_14,weight_15,weight_16,weight_17,weight_18,weight_19,weight_20,weight_21
0,age_1864,m,1000,30.705676,12.289888,56.887145,10.449594,38.612621,47.343966,8.789196,...,53.887440,51.003342,38.482515,57.421184,11.920063,34.641819,49.529351,29.615779,48.801957,14.259688
1,age_65,f,1001,16.498045,11.177411,17.671996,11.782094,25.103857,11.047300,7.847097,...,17.804158,11.059227,34.189175,14.226022,9.710651,24.793063,22.071087,14.110091,23.267020,8.170437
2,age_65,f,1002,16.498045,11.177411,17.671996,11.782094,25.103857,11.047300,7.847097,...,17.804158,11.059227,34.189175,14.226022,9.710651,24.793063,22.071087,14.110091,23.267020,8.170437
3,age_1864,m,1003,30.705676,12.289888,56.887145,10.449594,38.612621,47.343966,8.789196,...,53.887440,51.003342,38.482515,57.421184,11.920063,34.641819,49.529351,29.615779,48.801957,14.259688
4,age_1864,f,1004,24.616020,10.745798,43.986873,9.275901,32.214132,31.668398,7.801797,...,39.738820,34.878200,34.810354,40.644540,10.017054,29.802253,39.250416,23.135184,40.947464,11.286525


Now we need to test that these weights we've calculated are actually correct and make sense with what our constraints tell us.

In [26]:
# creating marginal distribution of individuals 
bsa_agg0 = constraints.apply(lambda x: 1.0*bsa_agg, axis=1)

bsa_agg_final = (bsa_agg0 * np.nan).copy()

for i in range(0,len(constraints)):
    bsa_agg_final.iloc[i] = bsa_categorical.apply(lambda x: x*BSA_2020_copy['weight_'+str(i)],axis=0).sum(axis=0)

bsa_agg_final # this comes out in a funny order so we'll swap the columns to match our constraints for easy comparison

,age_1864,age_65,f,m
0,5183.0,1553.0,3507.000000,3229.000000
1,2179.0,1000.0,1682.000000,1497.000000
2,9413.0,1700.0,5818.000000,5295.000000
3,1869.0,1045.0,1533.000000,1381.000000
4,6665.0,2308.0,4723.000000,4250.000000
5,7256.0,1163.0,4125.000000,4294.000000
6,1572.0,696.0,1211.000000,1057.000000
7,7092.0,1029.0,4024.000000,4097.000000
8,6979.0,2958.0,5331.000000,4606.000000
9,4371.0,212.0,2500.000000,2083.000000


In [27]:
bsa_agg_final_ordered = bsa_agg_final.reindex(columns=['m', 'f', 'age_1864', 'age_65'])
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.reindex.html
bsa_agg_final_ordered

,m,f,age_1864,age_65
0,3229.000000,3507.000000,5183.0,1553.0
1,1497.000000,1682.000000,2179.0,1000.0
2,5295.000000,5818.000000,9413.0,1700.0
3,1381.000000,1533.000000,1869.0,1045.0
4,4250.000000,4723.000000,6665.0,2308.0
5,4294.000000,4125.000000,7256.0,1163.0
6,1057.000000,1211.000000,1572.0,696.0
7,4097.000000,4024.000000,7092.0,1029.0
8,4606.000000,5331.000000,6979.0,2958.0
9,2083.000000,2500.000000,4371.0,212.0


In [28]:
constraints

,m,f,age_1864,age_65
0,3229,3507,5183,1553
1,1497,1682,2179,1000
2,5295,5818,9413,1700
3,1381,1533,1869,1045
4,4250,4723,6665,2308
5,4294,4125,7256,1163
6,1057,1211,1572,696
7,4097,4024,7092,1029
8,4606,5331,6979,2958
9,2083,2500,4371,212


The output from our marginal distribution matches our constraints, so we can conclude that the weights we've calculated are correct.

### Step 5: Intergerisation using the TRS method


In [29]:
# intergerisation (Lovelace and Ballas, 2013)
def int_trs(weights):
    xv = np.array(weights).ravel() # convert to vector if necessary
    xint = np.floor(xv) # truncate - get the integer part of the result
    r = xv - xint # get the decimal of the weight
    frac_sum = round(r.sum()) # work out deficit population
    xs = np.random.choice(len(xv),int(frac_sum),True,r/r.sum()) # sample based on deficit
    topup = np.bincount(xs,minlength=len(xv)) # result of the sample deficit
    return xint + topup

### Step 6: Expansion

In [30]:
# expansion
def int_expansion(weights):
    return np.repeat(range(0,len(weights)),weights.astype(int))

### Step 7: Putting the two previous steps together

In [31]:
# putting this all together!

individuals = [] # creating a list to hold the results of integerisation and expansion

for i in range(0, len(constraints)):
    # calls both functions in one line to integerise and expand
    integerise = int_expansion(int_trs(BSA_2020_copy['weight_' + str(i)]))
    current_ind = BSA_original.loc[integerise].copy() # Select the relevant individuals using .loc
    current_ind['zone'] = i     # Assigning the 'zone' - allocates individuals to different zones
    individuals.append(current_ind)

BSA_2020_synthetic = pd.concat(individuals) # concatonating the resulting list into a new dataframe
BSA_2020_synthetic.reset_index(drop=True, inplace=True) #Resetting the index

In [32]:
BSA_2020_synthetic # looking at our new synthetic population of York!

,id,age,sex,economic,partySup,partySupWho,polInterest,welfare,redistrb,leftright,leftright2,libauth,libauth2,religion,nationality,raceOrigin,disability,voteAct,voteParty,zone
0,1000,27,2,2,1,1.0,3,3,5,3.8,3,4.500000,3,5,1,3.0,3,1,1.0,0
1,1000,27,2,2,1,1.0,3,3,5,3.8,3,4.500000,3,5,1,3.0,3,1,1.0,0
2,1000,27,2,2,1,1.0,3,3,5,3.8,3,4.500000,3,5,1,3.0,3,1,1.0,0
3,1000,27,2,2,1,1.0,3,3,5,3.8,3,4.500000,3,5,1,3.0,3,1,1.0,0
4,1000,27,2,2,1,1.0,3,3,5,3.8,3,4.500000,3,5,1,3.0,3,1,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162091,1275,50,2,2,2,7.0,4,2,1,1.0,1,4.833333,3,5,2,3.0,3,1,7.0,21
162092,1275,50,2,2,2,7.0,4,2,1,1.0,1,4.833333,3,5,2,3.0,3,1,7.0,21
162093,1275,50,2,2,2,7.0,4,2,1,1.0,1,4.833333,3,5,2,3.0,3,1,7.0,21
162094,1275,50,2,2,2,7.0,4,2,1,1.0,1,4.833333,3,5,2,3.0,3,1,7.0,21


Now we'll add the geo_codes column back in to our new synthetic dataframe, so we have a spatial element that we can connect to our shapefile in order to perform our geodemographics!

In [33]:
constraints_geo_copy = constraints_geo.copy()
constraints_geo = constraints_geo.drop(columns=['m','f','age_1864','age_65'])
constraints_geo

,geo_code
0,E05001745
1,E05001746
2,E05001747
3,E05001748
4,E05001749
5,E05001750
6,E05001751
7,E05001752
8,E05001753
9,E05001754


In [34]:
BSA_synthetics_geo = pd.merge(BSA_2020_synthetic,constraints_geo, left_on='zone', right_index=True, validate='many_to_one')
    # (pandas, 2024)
    # https://pandas.pydata.org/docs/dev/reference/api/pandas.merge.html#pandas.merge

In [35]:
# let's drop the zone column too for good measyre
BSA_synthetics_geo = BSA_synthetics_geo.drop(columns=['zone'])
BSA_synthetics_geo

,id,age,sex,economic,partySup,partySupWho,polInterest,welfare,redistrb,leftright,leftright2,libauth,libauth2,religion,nationality,raceOrigin,disability,voteAct,voteParty,geo_code
0,1000,27,2,2,1,1.0,3,3,5,3.8,3,4.500000,3,5,1,3.0,3,1,1.0,E05001745
1,1000,27,2,2,1,1.0,3,3,5,3.8,3,4.500000,3,5,1,3.0,3,1,1.0,E05001745
2,1000,27,2,2,1,1.0,3,3,5,3.8,3,4.500000,3,5,1,3.0,3,1,1.0,E05001745
3,1000,27,2,2,1,1.0,3,3,5,3.8,3,4.500000,3,5,1,3.0,3,1,1.0,E05001745
4,1000,27,2,2,1,1.0,3,3,5,3.8,3,4.500000,3,5,1,3.0,3,1,1.0,E05001745
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162091,1275,50,2,2,2,7.0,4,2,1,1.0,1,4.833333,3,5,2,3.0,3,1,7.0,E05001766
162092,1275,50,2,2,2,7.0,4,2,1,1.0,1,4.833333,3,5,2,3.0,3,1,7.0,E05001766
162093,1275,50,2,2,2,7.0,4,2,1,1.0,1,4.833333,3,5,2,3.0,3,1,7.0,E05001766
162094,1275,50,2,2,2,7.0,4,2,1,1.0,1,4.833333,3,5,2,3.0,3,1,7.0,E05001766


Now we have complete synthetic population representing BSA results for the whole of York, and including geo_code zones that we can use to map the data. We'll save our new dataframe as a CVS at this point, and that is the spatial microsimulation part of the project all done!

In [36]:
BSA_synthetics_geo.to_csv("data/BSA_synthetics_geo.csv", index=False)

## Example of results